In [1]:
import boto3
from boto3 import client
import json
import pandas as pd

In [2]:
s3 = boto3.resource('s3')

In [37]:
pc_bucket = s3.Bucket('canopy-production-ml')

In [52]:
chips = []
for obj in pc_bucket.objects.all():
    if 'yes' in obj.key:
        chips.append(obj.key)

In [15]:
for chip in chips[0:4]:
    id = chip.key.split("/")[4]
    

100
100
100
100


In [55]:
j_file = "train_test_polygons.json"

with open(j_file, 'r') as j:
    train_test = json.loads(j.read())


In [56]:
len(train_test["train"])

46

In [57]:
len(train_test["test"])

51

In [53]:
chips[0]

'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3000.tif'

# Export Chip List

In [54]:
import json

with open('yes_chips_s3.json', 'w') as fp:
    json.dump(chips, fp)


In [58]:
chips[0:3]

['chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3000.tif',
 'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3100.tif',
 'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_400.tif']

# Train / Test Copy

In [59]:
def train_test_s3_copy(chips, j_file, bucket_name='canopy-production-ml',
                       base_path='chips/cloudfree-merge-polygons/split/'):
    
    bucket = s3.Bucket(bucket_name)
    
    with open(j_file, 'r') as j:
        
        train_test_file = json.loads(j.read())
        
#     for split in list(train_test.keys()):
        
#         for polygon in train_test[split]:

    length = len(chips)
            
    for i, chip in enumerate(chips, 1):
        print(f'Processing chip {i} of {length}', end='\r', flush=True)

        CopySource = {
            'Bucket': bucket_name,
            'Key': chip
        }

        polygon_id = int(chip.split("/")[5])

        filename = chip.split('/')[-1]

        if polygon_id in train_test_file["test"]:
            train_test = 'test'

            #s3.Object(bucket, base_path + 'test/').copy_from(CopySource=bucket + chip.key)

        else:
            train_test = 'train'

            #s3.Object(bucket, base_path + 'train_val/').copy_from(CopySource=bucket + chip.key)

        new_key = f'{base_path}{train_test}/{polygon_id}/{filename}'

        bucket.copy(CopySource, new_key)

In [61]:
j_file = "train_test_polygons.json"
train_test_s3_copy(chips,j_file)

# Train / Val Split

In [7]:
def s3_list(dir_name='train',bucket='canopy-production-ml'):
    
    pc_bucket = s3.Bucket(bucket)
    
    train_chips = []

    for obj in pc_bucket.objects.all():
        
        if dir_name in obj.key:
            train_chips.append(obj)
            
    return train_chips
    

In [8]:
train_chips = s3_list()

# val_chips = s3_list(dir_name='val')

In [9]:
train_chips[0:3]

[s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/'),
 s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/1/1_1000_1000.tif'),
 s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/1/1_1000_1100.tif')]

In [93]:

def train_val_split(train_chips,bucket_name='canopy-production-ml',
                    in_base_path='chips/cloudfree-merge-polygons/split/',
                    out_base_path='chips/cloudfree-merge-polygons/split/val/'):
    
    bucket = s3.Bucket(bucket_name)
    
    train_chips = train_chips.copy()
    
    dir_dict = {}
    
    for chip_1 in train_chips:
        
        chip_key = chip_1.key
        
        poly_id = chip_key.split("/")[-1].split("_")[0]
        
        if poly_id in dir_dict.keys():
                
            dir_dict[poly_id].append(chip_key)
                
        else:

            dir_dict[poly_id] = [chip_key]
            
    
    key_num = len(dir_dict.keys())
    
    for i_1,key in enumerate(dir_dict.keys(),1):
        
        dir_len = len(dir_dict[key])
        
        val_slice = dir_len // 5 
        
        val_chips = dir_dict[key][:val_slice]
        
        val_num = len(val_chips)
        
        for i_2,val_chip_key in enumerate(val_chips,1):
            
            print(f"Copying chip {i_2} of {val_num} in polygon {i_1} of {key_num}", end='\r', flush=True)
            
            CopySource = {
                'Bucket': bucket_name,
                'Key':val_chip_key}
            
            polygon_id = val_chip_key.split("/")[-1].split("_")[0]
            
            filename = val_chip_key.split('/')[-1]
            
            new_key = f'{out_base_path}{polygon_id}/{filename}'
            
            bucket.copy(CopySource, new_key)
            
            obj = s3.Object('canopy-production-ml', val_chip_key)
            
#             obj.delete()
    

In [94]:
train_val_split(train_chips)

In [81]:
val_chips = s3_list(dir_name='val',bucket='canopy-production-ml')

In [95]:
len(train_chips)

34071

In [12]:
def copy_dir(file_list,
             out_base_path='chips/cloudfree-merge-polygons/split/train_copy/',
             bucket_name='canopy-production-ml'):

    bucket = s3.Bucket(bucket_name)

    dir_num = len(file_list)

    for i_2,chip in enumerate(file_list,1):

        chip_key = chip.key

        print(f"Copying chip {i_2} of {dir_num}", end='\r', flush=True)

        CopySource = {
            'Bucket': 'canopy-production-ml',
            'Key':chip_key}

        polygon_id = chip_key.split("/")[-1].split("_")[0]

        filename = chip_key.split('/')[-1]

        new_key = f'{out_base_path}{polygon_id}/{filename}'

        bucket.copy(CopySource, new_key)

In [ ]:
copy_dir(train_chips[1:])

In [33]:
for chip in val_chips[1:]:
    
    obj = s3.Object('canopy-production-ml', chip.key)
    obj.delete()

# Convert to Lst Format 

In [5]:
train_test_dict = {"train": 
                   [12, 74, 7, 28, 10, 19, 9, 21, 32, 31, 64, 58, 86, 27, 77, 53, 33, 84, 30, 18, 22, 15, 25, 78, 75, 85, 89, 55, 39, 66, 44, 59, 81, 60, 92, 101], 
                   "test": [1, 6, 2, 72, 70, 14, 20, 5, 13, 26, 3, 11, 4, 37, 76, 69, 41, 35, 47, 46, 43, 34, 63, 61, 17, 23, 16, 24, 8, 48, 29, 68, 36, 83, 88, 42, 40, 38, 45, 71, 87, 49, 52, 82, 57, 50, 56, 80, 51, 54, 91, 99, 100, 93, 90, 94, 67, 65, 62, 73, 79]}

In [4]:
df_in = pd.read_csv("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Labels/local_multi_label.csv")

In [7]:
def convert_to_lst(df_in):
    # insert code
    return
    
    
    
    
    